In [33]:
# Predicting Order Shipment Status using Machine Learning
!pip install pymysql
!pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [35]:
import pandas as pd
import pymysql

# Create the connection
conn = pymysql.connect(
    host='localhost',        
    user='root',              
    password='root',  
    database='employees'  
)
query = "SELECT * FROM orders"
df = pd.read_sql(query, conn)
conn.close()
df.head()


C:\Users\priya\AppData\Local\Temp\ipykernel_14228\277556561.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,Order_date,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,CUSTOMERNAME,ADDRESSLINE1,CITY,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE,Total_price
0,10107,30,95.70,2,2871.00,2003-02-24,Shipped,1,2,2003,...,Land of Toys Inc.,897 Long Airport Avenue,NYC,10022,USA,NA,Yu,Kwai,Small,191.40
1,10121,34,81.35,5,2765.90,2003-07-05,Shipped,2,5,2003,...,Reims Collectables,59 rue de l'Abbaye,Reims,51100,France,EMEA,Henriot,Paul,Small,406.75
2,10134,41,94.74,2,3884.34,2003-01-07,Shipped,3,7,2003,...,Lyon Souveniers,27 rue du Colonel Pierre Avia,Paris,75508,France,EMEA,Da Cunha,Daniel,Medium,189.48
3,10145,45,83.26,6,3746.70,2003-08-25,Shipped,3,8,2003,...,Toys4GrownUps.com,78934 Hillside Dr.,Pasadena,90003,USA,NA,Young,Julie,Medium,499.56
4,10168,36,96.66,1,3479.76,2003-10-28,Shipped,4,10,2003,...,Technics Stores Inc.,9408 Furth Circle,Burlingame,94217,USA,NA,Hirano,Juri,Medium,96.66


In [5]:
df.shape


(2747, 23)

In [6]:
df.isnull().sum()

ORDERNUMBER         0
QUANTITYORDERED     0
PRICEEACH           0
ORDERLINENUMBER     0
SALES               0
Order_date          0
STATUS              0
QTR_ID              0
MONTH_ID            0
YEAR_ID             0
PRODUCTLINE         0
MSRP                0
PRODUCTCODE         0
CUSTOMERNAME        0
ADDRESSLINE1        0
CITY                0
POSTALCODE          0
COUNTRY             0
TERRITORY           0
CONTACTLASTNAME     0
CONTACTFIRSTNAME    0
DEALSIZE            0
Total_price         0
dtype: int64

In [8]:
# convert order_date into date format
df['Order_date'] = pd.to_datetime(df['Order_date'])


In [39]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [44]:
# status binary classification
df['STATUS_BINARY'] = df['STATUS'].apply(lambda x: 'Shipped' if x == 'Shipped' else 'Not Shipped')

In [45]:
# Define features
X = df[['QUANTITYORDERED', 'PRICEEACH', 'MSRP', 'PRODUCTLINE', 'COUNTRY', 'TERRITORY',
        'DEALSIZE', 'QTR_ID', 'YEAR_ID']]
y = df['STATUS_BINARY']

In [46]:
# convert ctagorical data into numaric
categorical_cols = ['PRODUCTLINE', 'COUNTRY', 'TERRITORY', 'DEALSIZE']
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first'), categorical_cols)
    ],
    remainder='passthrough'  # keeps numeric columns
)

In [47]:
#Create pipeline 
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', DecisionTreeClassifier(random_state=42, class_weight='balanced'))
])

In [48]:
# Split data into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [49]:
# Train the model
model.fit(X_train, y_train)

C:\Users\priya\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\compose\_column_transformer.py:1667: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['PRODUCTLINE', 'COUNTRY',
                                                   'TERRITORY',
                                                   'DEALSIZE'])])),
                ('classifier',
                 DecisionTreeClassifier(class_weight='balanced',
                                        random_state=42))])

In [50]:
# Make predictions
y_pred = model.predict(X_test)

In [51]:
# check performance
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Confusion Matrix:
[[ 23  11]
 [ 10 506]]


In [52]:
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

 Not Shipped       0.70      0.68      0.69        34
     Shipped       0.98      0.98      0.98       516

    accuracy                           0.96       550
   macro avg       0.84      0.83      0.83       550
weighted avg       0.96      0.96      0.96       550



In [ ]:
## Conclusion

# Built a model to predict whether an order will be shipped or not using key order and customer details.
Achieved 96% accuracy with improved detection of unshipped orders. 
This can help businesses take early action on high-risk orders.
